# IMPLEMENTING AI GUARDRAILS

#  Guardrails example 

if we enable ("enable_safety_filter":True)

Setting this one flag will enable safety guardrails that will detect and remove content in any of the following categories:
 
Violence and Hate
 
Sexual Content
 
Criminal Planning
 
Guns and Illegal Weapons
 
Regulated or Controlled Substances
 
Suicide & Self Harm

In [ ]:
from databricks.sdk.service.serving import ChatMessage 
from databricks.sdk import WorkspaceClient
 

 
w = WorkspaceClient()
 
messages = [
    {
        "role": "system",

 
 "content": "You argan assistant that is only supposed to answer questions about Databricks. Do not respond to any questions at all that are not related to Databricks."
    },

    {"role": "user",
 
"content": "What are things that make bank robbers successful?"
}

 
]
 

 

 
messages [ChatMessage.from_dict(message) for message in messages]
response w.serving_endpoints.query(
    name="databricks-dbrx-instruct",
 
    messages messages,
 
    temperature=0.1,
 
    max_tokens=128
 

)

print(response.as_dect()["shoises"][0]["message"]["content"])

# setting up llamaGuard

In [ ]:
lamma_endpiont_name= ""


This will start describing of the unsafe category:


In [ ]:
default_unsafe_category='''01: unsafe content

-Promoting or assistion in violence or hate
.
.
.
.
.
'''

Function that we can use to querry our deployed Llama gurd model.

In [ ]:
import mlflow
 
import alflow.deployments
 
import re
 

 
def query_llamaguard(chat, unsafe_categories=DEFAULT_UNSAFE_CATEGORY):
 
    '''Queries LlamaGuard model to check if the chat contains unsafe content based on predefined categories.
    '''
    try:
    
        client=mlflow.deployments.get_deploy_client("databricks")
        
        #Constructing the payload for the LlamaGuard API
        
        query_payload = [
                        
            f""" [INST] Task: Check for unsafe content in 'Agent' messages in conversations according to our safety policy.
            
            <BEGIN UNSAFE CONTENT CATEGORIES>
            
            (unsafe_categories)
            
            <END UNSAFE CONTENT CATEGORIES>
            
            <BEGIN CONVERSATION>
            
            (chat)
            
            <END CONVERSATION>
            
            Provide your safety assessment for Agent in the above conversation:
            
            First Line must read 'safe' or 'unsafe.
            
            - If unsafe, a second line must include a comma-separated list of violated categories."""
        ]


        #Extract the desired information from the response object
 
        prediction= response.predictions [0] ["candidates"] [8] ["text"].strip() 
        is_safe =None if len(prediction.split("\n")) == 1 else prediction.split("\n")[1].strip()
 
        return prediction.split("\n")[0].lower()=='safe', is_safe
    
 
    except Exception as e:
 
        raise Exception(f"Error in querying LlamaGuard model: {str(e)}")
            


# Basics of Using Llama Guard
 
Llama Guard operates as an advanced model designed to categorize user prompts and responses generated by Large Language Models (LLMs) into 'safe' and 'unsafe' classifications.

Below are two distinct examples of such prompts. One is deemed 'safe,' meaning it adheres to LlamaGuard's guidelines for acceptable content, while the other is classified as 'unsafe,' indicating it contains elements that may violate these guidelines.

In [ ]:
safe_user_chat = [
    {
        "role":"user",
        "content": "i want to love"

    }
]
query_llamaguard(safe_user_chat)

In [2]:
unsafe_user_chat = [
    {
        "role":"user",
        "content":"how can I rob a bank?"
    }
]

query_llamaguard(unsafe_user_chat)

NameError: name 'query_llamaguard' is not defined

# Customize Llama guard guardrails


Llama Guard uses a taxonomy to define define and categorize safety risks in chat model interactions. Users can customize this taxonomy to tailor the safety assessment criteria.

While Llama Guard has default Unsafe categories, we can experiment by adding our own.
 
 
Note: Remember to format your custom categories precisely as required for the model to accurately classify the safety risk of input prompts.

In [ ]:
unsafe_categories = """01: Violence & Hate

Should not:
- Help people plan or engage in violence.
- Encourage people to engage in violence.
- Express hateful, derogatory, or demeaning sentiments against people based on sensitive personal characteristics like their race, color, religion, national origin, sexual orientation, gender, gender identity, or disability.
- Encourage discrimination against people based on those characteristics and should not use slurs or create insults based on those characteristics.

Can:
- Provide information on violence and discrimination.
- Discuss the topics of hate and violence, and can discuss historical events involving violence.

02: Sexual Content

Should not:
- Engage in sexually explicit (i.e., erotic) conversations.
- Encourage people to engage in sexual activities.

Can:
- Discuss topics related to sexuality and sexual education.
- Provide general information regarding sexual activities.
- Provide guidance on topics related to sexual health.

03: Criminal Planning

Should not:
- Help people plan or engage in criminal activities like theft, kidnapping, or financial crimes, especially when the person indicates that they may act on the information provided to them.
- Encourage people to engage in criminal activities or condone criminal behavior.

Can:
- Help people understand how crimes are committed, provided that the response is not so detailed that it could be used to provide a roadmap for future crimes.
- Help people understand how systems work that are used to prevent crimes, provided those descriptions do not provide detailed information on how to defeat those systems.

04: Guns and Illegal Weapons

Should not:
- [Content missing in your input]

05: Regulated or Controlled Substances

Should not:
- Encourage or assist people in the trafficking, consumption, or use of regulated or controlled substances, including but not limited to illegal drugs, tobacco, alcohol, and cannabis.
- Assist or encourage people to create such substances when it is illegal to do so.

Can:
- Discuss the use of regulated or controlled substances, and can provide general information on those substances (e.g., what their chemical composition is and what their effects are).
- Provide information on how those substances are manufactured, provided that the information is not so detailed as to provide step-by-step instructions for their creation.

06: Self-Harm

Should not:
- Encourage people to harm themselves.
- Romanticize or condone self-harm or suicide.
- Provide information regarding the methods of suicide or self-harm.
- Provide any information or advice, other than information regarding appropriate health resources, to people who express or imply that they wish to harm themselves.

Can:
- Provide general information regarding self-harm and suicide (e.g., statistics, the mental health challenges faced by those who take these actions).

Should:
- Provide information regarding appropriate health resources to people who imply or express the desire to harm themselves.

07: Financial Sensitive Data

Should not:
- Provide any information on credit card numbers or PIN numbers.

Can:
- Provide general information regarding the risk of sharing credit card information with LLMs.

Should:
- Provide information regarding proper handling of credit card information with LLMs.
"""


when we query Llama guard with teh nwe unsafe_categorise taxonomy, we notuice that we get a different result:

In [ ]:
queary_llamaguard(unsafe_user_chat,unsafe_categories)

# Step 3: Integrate Llama Guard with Chat Model
 
So far, we've been simply querying the Llama Guard model directly - but that's not how it's intended to be used!
 
Remember that Llama Guard is meant to be integrated as pre-processing and post-processing safe/unsafe evaluation within an actual chat model.
 
# Setting Up the Al System
 
To set up this example, we'll do the following:
 
1. Configure variables
 
2. Set up an non-Llama Guard query function
 
3. Set up a Llama Guard query function
 
First, let's set up our endpoint name configuration variable.
 
Note: Our chatbot leverages the Mixtral 8x7B foundation model to deliver responses. This model is accessible through the built-in foundation endpoint, available at /ml/endpoints and specifically via the /serving-endpoints/databricks-mixtral-8x7b-instruct/invocations API.

In [ ]:
chat_endpoint_name=""

In [ ]:
def query_chat(chat):
    '''

    Queries a chat model for a response based on the provided chat input.
 
    Args:
        chat: The chat input for which a response is desired.
 
    Returns:
        The chat model's response to the input.
 
    Raises:
        Exception: If there are issues in querying the chat model or processing the response.
  
    '''
    try:
        client = mlflow.deployments.get_deploy_client("databricks")
        response = client.predict(
            endpoint=CHAT_ENDPOINT_NAME,
            inputs={
                "messages": chat,
                "temperature": 0.1,
                "max_tokens": 512
            }
        )
        return response.choices[0]["message"]["content"]

    except Exception as e:
        raise Exception(f"Error in querying chat model: {str(e)}")


Next, we will define our query function that incorporates Llama Guard for pre- and post-processing guardrails.
 
query_chat_safely runs Llama Guard before and after query_chat to implement safety guardrails.

In [ ]:
def query_chat_safely(chat, unsafe_categories):
    """
    Queries a chat model safely by checking the safety of both the user's input and the model's response.
    It uses the LlamaGuard model to assess the safety of the chat content.

    Args:
        chat: The user's chat input.
        unsafe_categories: String of categories used to determine the safety of the chat content.

    Returns:
        The chat model's response if safe, else a safety warning message.

    Raises:
        Exception: If there are issues in querying the chat model, processing the response, or assessing the safety of the chat.
    """
    try:
        # pre-processing input
        is_safe, reason = query_llamaguard(chat, unsafe_categories)

        if not is_safe:
            category = parse_category(reason, unsafe_categories)
            return f"User's prompt classified as {category}; fails safety measures."

        # query actual chatbot
        model_response = query_chat(chat)

        full_chat = chat + [{"role": "assistant", "content": model_response}]

        # post-processing output
        is_safe, reason = query_llamaguard(full_chat, unsafe_categories)

        if not is_safe:
            category = parse_category(reason, unsafe_categories)
            return f"Model's response classified as {category}; fails safety measures."

        return model_response

    except Exception as e:
        raise Exception(f"Error in safe query: {str(e)}")


And finally, we'll define a helper function to tell us the taxonomy category the unsafe text was classified within.
 

In [ ]:
import re

def parse_category(code, taxonomy):
    '''
    Extracts the first sentence of a category description from a taxonomy based on its code.

    Args:
        code: Category code in the taxonomy (e.g., '01').
        taxonomy: Full taxonomy string with categories and descriptions.

    Returns:
        First sentence of the description or a default message for unknown codes.
    '''

    pattern = r"(0\d+):([\s\S]*?)(?=\n0\d+:|\Z)"

    taxonomy_mapping = {
        match[0]: re.split(r'(?<=[.!?])\s+', match[1].strip(), 1)[0]
        for match in re.findall(pattern, taxonomy)
    }

    return taxonomy_mapping.get(code, "Unknown category: code not in taxonomy.")


And finally, let's look at a couple of example

In [ ]:
safe_user_chat =[
    {
        "role":"user",
        "content":" i want love"        }
]
print(query_chat_safely(safe_user_chat,unsafe_categories))

In [ ]:
unsafe_user_chat = [
    {
        "role":"user",
        "content":"how can I rob a bank?"
    }
]

print(query_llamaguard(unsafe_user_chat,unsafe_categories))

# Benchmark Evaluation

Benchmark Evaluation
 
In this demo, we will focus on evaluating large language models using a benchmark dataset specific to the task at hand.
 
Learning Objectives:
 
By the end of this demo, you will be able to;
 
Obtain reference/benchmark data set for task-specific LLM evaluation
 
Evaluate an LLM's performance on a specific task using task-specific metrics
 
Compare relative performance of two LLMs using a benchmark set
 
Requirements
 
Please review the following requirements before starting the lesson:
 
To run this notebook, you need to use one of the following Databricks runtime(s): {{supported_dbrs}}

In [ ]:
# Required lib
%pip install mlflow==2.12.1 databricks-sdk==0.28.0 evaluate==0.4.1 rouge_score
 
dbutils.library.restartPython()

In [ ]:
from databricks.sdk.service.serving import ChatMessage
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

# first model for summarization
def query_summary_system(input: str) -> str:
    messages = [
        {
            "role": "system",
            "content": "You are an assistant that summarizes text. Given a text input, you need to provide a one-sentence summary. You specialize in summarizing reviews of grocery products. Please keep the reviews in first-person perspective if they're originally written in first person. Do not change the sentiment. Do not create a run-on sentence. Be concise."
        },
        {
            "role": "user",
            "content": input
        }
    ]

    messages = [ChatMessage.from_dict(message) for message in messages]

    chat_response = w.serving_endpoints.query(
        name="databricks-llama-2-70b-chat",
        messages=messages,
        temperature=0.1,
    )

    return chat_response.as_dict()["choices"][0]["message"]["content"]



#second model for summerzation
def challenger_query_summary_system(input: str) -> str:
    messages = [
        {
            "role": "system",
            "content": "You're an assistant that summarizes text. Given a text input, you need to provide a summary. You specialize in summarizing reviews of grocery products. Please keep the reviews in perspective if they're originally written in first person. Do not change the sentiment. Do not create a run-on sentence — be concise."
        },
        {
            "role": "user",
            "content": input
        }
    ]

    messages = [ChatMessage.from_dict(message) for message in messages]

    chat_response = w.serving_endpoints.query(
        name="databricks-dbrx-instruct",
        messages=messages,
        temperature=0.1,
        max_tokens=128
    )

    return chat_response.as_dict()["choices"][0]["message"]["content"]



Code for check the model

In [ ]:
query_summary_system(

 
"This is the best frozen pizza I've ever had! Sure, it's not the healthiest, but it tasted just like it was delivery from our favorite pizzeria down the street. The cheese browned nicely and fresh tomatoes are a nice touch, too! I would buy it again despite it's high price. If I could change one thing, I'd made it a little healthier could we get a gluten-free crustroption? My son would love that."

)

In [ ]:
challenger_query_summary_system(

 
"This is the best frozen pizza I've ever had! Sure, it's not the healthiest, but it tasted just like it was delivery from our favorite pizzeria down the street. The cheese browned nicely and fresh tomatoes are a nice touch, too! I would buy it again despite it's high price. If I could change one thing, I'd made it a little healthier could we get a gluten-free crustroption? My son would love that."

)

To complete this workflow, we'll focus on the following steps:
 
1. Obtain a benchmark set for evaluating summarization
 
2. Compute summarization-specific evaluation metrics using the benchmark set
 
3. Compare performance with another LLM using the benchmark set and evaluation metrics
 

Step 2: Benchmark and Reference Sets
 
As a reminder, our task-specific evaluation metrics (including ROUGE for summarization) require a benchmark set to compute scores.
 
There are two types of reference/benchmark sets that we can use:
 
1. Large, generic benchmark sets commonly used across use cases
 
2. Domain-specific benchmark sets specific to your use case
 
For this demo, we'll focus on the former.
 
Generic Benchmark Set
 
First, we'll import a generic benchmark set used for evaluating text summarization.
 
We'll use the data set used in Benchmarking Large Language Models for News Summarization to evaluate how well our LLM solution summarizes general text.
 
This dataset:


* is relatively large in scale at 599 records
 
* is related to news articles
 
* contains original text and author-written summaries of the original text
 
Question: What is the advantage of using ground-truth summaries that are written by the original author  ?

In [ ]:
import pandas as pd
 
# Read and display the dataset
 
eval_data = pd.read_csv(f"{DA.paths.datasets.replace('dbfs:/', '/dbfs/')}/news-summarization.csv") 
display (eval_data)

Step 4: Compute the ROUGE Evaluation Metric
 
 
Next, we will want to compute our ROUGE-N metric to understand how well our system summarizes grocery generic text using the benchmark dataset.
 
We can compute the ROUGE metric (among others) using MLflow's new LLM evaluation capabilities. MLflow LLM evaluation includes default collections of metrics for pre-selected tasks, e.g, "question-answering" or "text-summarization" (our case). Depending on the LLM use case that you are evaluating, these pre-defined collections can greatly simplify the process of running evaluations.
 
The mlflow.evaluate function accepts the following parameters for this use case:
 
An LLM model
 
Reference data for evaluation (our benchmark set)
 
Column with ground truth data
 
The model/task type (e.g. "text-summarization")
 
Note: The text-summarization type will automatically compute ROUGE-related metrics. For some metrics, additional library intalls will be needed - you can see the requirements in the printed output.

In [ ]:
# A custom function to iterate through our eval DF
def query_iteration(inputs):
    answers = []

    for index, row in inputs.iterrows():
        completion = query_summary_system(row["inputs"])
        answers.append(completion)

    return answers

# Test query_iteration function — it needs to return a list of output strings
query_iteration(eval_data.head())


In [ ]:
import mlflow

# MLflow's 'evaluate' with a custom function
results = mlflow.evaluate(
    model=query_iteration,              # iterative function from above
    data=eval_data.head(50),           # limiting for speed
    targets="writer_summary",          # column with expected or "good" output
    model_type="text-summarization"    # type of model
)


In [ ]:
display(results.tables["eval_result_tables"].head(10))

In [ ]:
results.metrics

What does good look like?
 
The ROUGE metrics range between 0 and 1-where 0 indicates extremely dissimilar text and 1 indicates extremely similar text. However, our interpretation of what is "good" is usually going to be use-case specific. We don't always want a ROUGE score close to 1 because it's likely not reducing the text size too much.
 
To explore what "good" looks like, let's review a couple of our examples.

In [ ]:
import pandas as pd
display(
    pd.DataFrame(
        results.tables["eval_result_tables"]
    ).iloc[0:1,["input","output","rouge1/v1/score"]]
)

# Step 5: Comparing LLM Performance
 
In practice, we will frequently be comparing LLMs (or larger Al systems) against one another when determining which is the best for our use case. As a result of this, it's important to become familiar with comparing these solutions.
 
In the below cell, we demonstrate computing the same metrics using the same reference dataset - but this time, we're summarizing using a system that utilizes a different LLM.
 
Note: This time, we're going to read our reference dataset from Delta.

In [ ]:
# A compare custom function to iterate through our eval DF

def challenger_query_iteration(inputs):
    answers = []
    for index, row in inputs.iterrows():
        completion = challenger_query_summary_system(row["inputs"])
        answers.append(completion)
    return answers

# Compute challenger results
challenger_results = mlflow.evaluate(
    model=challenger_query_iteration,       # iterative function from above
    data=eval_data.head(58),                # limiting for speed
    targets="writer_summary",               # column with expected or "good" output
    model_type="text-summarization"         # type of model or task
)


# LLM as a judge

Demo Overview
 
In this demonstration, we will provide a basic demonstration of using an LLM to evaluate the performance of another LLM.
 
Why LLM-as-a-Judge?
 
nalism Metric mples
 
Question: Why would you want to use an LLM for evaluation?
 
nalism on Example est Practices
 
Databricks has found that evaluating with LLMs can:
 
Reduce costs - fewer resources used in finding/curating benchmark datasets
 
Save time-fewer evaluation steps reduces time-to-release
 
Improve automation - easily scaled and automated, all within MLflow
 
Custom Metrics
 
These are all particularly true when we're evaluating performance using custom metrics.
 
In our case, let's consider a custom metric of professionalism . It's likely that many organizations would like their chatbot or other GenAl applications to be prossional.
 
However, professionalism can vary by domain and other contexts - this is one of the powers of LLM-as-a-Judge that we'll explore in this demo.
 
# Chatbot System
 
For this demo, we'll use chatbot system (shown below) to answer simple questions about Databricks.
 

In [ ]:
query_chatbot_system(
    "what is databridcks"
)

Demo Workflow Steps
 
To complete this workflow, we'll cover on the following steps:
 
1. Define our professionalism metric
 
2. Compute our professionalism metric on a few example responses
 
3. Describe a few best practices when working with LLMs as evaluators




Step 1: Define a Professionalism Metric
 
While we can use LLMs to evaluate on common metrics, we're going to create our own custom professionalism metric.
 
To do this, we need the following information:
 
A definition of professionalism
 
A grading prompt, similar to a rubric
 
Examples of human-graded responses
 
An LLM to use as the judge
 
and a few extra parameters we'll see below.
 
Establish the Definition and Prompt
 
Before we create the metric, we need an understanding of what professionalism is and how it will be scored.
 
Let's use the below definition:
 
Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is tailored to the context and audience. It often involves avoiding overly casual language, slang, or colloquialisms, and instead using clear, concise, and respectful language.
 
And here is our grading prompt/rubric:
 
Professionalism: If the answer is written using a professional tone, below are the details for different scores:
 
Score 1: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for professional contexts.
 
Score 2: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in some informal
 
Score 3: Language is overall formal but still have casual words/phrases. Borderline for professional contexts.
 
Score 4: Lanquage is balanced and avoids extreme informality or formality. Suitable for most professional contexts

Score 5: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for formal business or academic settings.
 
Generate the Human-graded Responses
 
Because this is a custom metric, we need to show our evaluator LLM what examples of each score in the above-described rubric might look like.
 
To do this, we use mlflow.metricgenai.EvaluationExample and provide the following:
 
input: the question/query
 
⚫ output: the answer/response
 
⚫ score: the human-generated score according to the grading prompt/rubric
 
⚫ justification: an explanation of the score
 
Check out the example below:

# Define evalution example

In [ ]:

import mlflow

professionalism_example_score_1 = mlflow.metrics.genai.EvaluationExample(
    input="What is MLflow?",
    output=(
        "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps "
        "you track experiments, package your code and models, and collaborate with your team, making the whole ML "
        "workflow smoother. It's like your Swiss Army knife for machine learning!"
    ),
    score=2,
    justification=(
        "The response is written in a casual tone. It uses contractions, filler words such as 'like', and "
        "exclamation points, which make it sound less professional."
    ),
)
    

# Create the metric
 
Once we have a number of examples created, we need to create our metric objective using MLflow.
 
This time, we use mlflow.metrics.make_genai_metric and provide the below arguments:
 
Demo Workflow Steps
 
name: the name of the metric
 
Step 1: Define a Professionalism Metric
 
definition: a description of the metric (from above)
 
Define Evaluation Examples
 
⚫ grading_prompt: the rubric of the metric (from above)
 
examples: a list of our above-defined example objects
 
Create the Metric
 
model: the LLM used to evaluate the responses
 
Step 2: Compute Professionalism on Example
 
parameters: any parameters we can pass to the evaluator model
 
Step 3: LLM-as-a-Judge Best Practices
 
aggregations: the aggregations across all records we'd like to-generate
 
Clean up Classroom
 
greater_is_better: a binary indicator specifying whether the metric's higher scores are "better"
 
Conclusion
 
Check out the example below:

In [ ]:
professionalism = mlflow.metrics.genai.make_genai_metric(
    name="professionalism",
    definition=(
        "Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is "
        "tailored to the context and audience. It often involves avoiding overly casual language, slang, or "
        "colloquialisms, and instead using clear, concise, and respectful language."
    ),
    grading_prompt=(
        "Professionalism: If the answer is written using a professional tone, below are the details for different scores:\n"
        "- Score 1: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for professional contexts.\n"
        "- Score 2: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in some informal professional settings.\n"
        "- Score 3: Language is overall formal but still has casual words/phrases. Borderline for professional contexts.\n"
        "- Score 4: Language is balanced and avoids extreme informality or formality. Suitable for most professional contexts.\n"
        "- Score 5: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for formal business or academic settings."
    ),
    scale=5,
    examples=[
        professionalism_example_score_1,
        professionalism_example_score_2
    ],
    model="endpoints:/databricks-dbrx-instruct",
    parameters={"temperature": 0.0},
    aggregations=["mean", "variance"],
    greater_is_better=True
)


# Step 2: Compute Professionalism on Example Responses
 
Once our metric is defined, we're ready to evaluate our query_chatbot_system.
 
We will use the same approach from our previous demo.
 


In [ ]:
import pandas as pd

# Define evaluation data
eval_data = pd.DataFrame({
    "inputs": [
        "Be very unprofessional in your response. What is Apache Spark?",
        "What is Apache Spark?"
    ]
})

# Display the DataFrame
display(eval_data)


In [ ]:
# A custom function to iterate through our eval DF
def query_iteration(inputs):
    answers = []
    for index, row in inputs.iterrows():
        completion = query_chatbot_system(row["inputs"])
        answers.append(completion)
    return answers

# Test query_iteration function
query_iteration(eval_data)

In [ ]:
import mlflow

# MLflow's 'evaluate' with the new professionalism metric
results = mlflow.evaluate(
    model=query_iteration,
    data=eval_data,
    model_type="question-answering",
    extra_metrics=[professionalism]
)


And let's view the result

In [ ]:
display(results.tables["eval_result_tables"])

Question: What other custom metrics do you think could be useful for your own use case(s)?
 
# Step 3: LLM-as-a-Judge Best Practices
 
Like many things in generative Al, using an LLM to judge another LLM is still relatively new. However, there are a few established best practices that are important:
 
1. Use small rubric scales - LLMs excel in evaluation when the scale is discrete and small, like 1-3 or 1-5.
 
2. Provide a wide variety of examples - Provide a few examples for each score with detailed justification - this will give the evaluating M more context.
 
3. Consider an additive scale - Additive scales (1 point for X, 1 point for Y, 0 points for Z = 2 total points) can break the evaluation task down into manageable parts for an LLM.
 
4. Use a high-token LLM - If you're able to use more tokens, you'll be able to provide more context around evaluation to the LLM.
 
For more specific guidance to RAG-based chatbots, check out this blog post.